In [58]:
import pandas as pd
import numpy as np
import psycopg2
import pandas as pd
from dotenv import load_dotenv
import os
import sys
from sqlalchemy import create_engine
from importlib import reload


In [62]:
import os, sys
from importlib import reload

MODULES_DIR = os.path.join(os.getcwd(), "..", "..", "modules")
sys.path.append(os.path.abspath(MODULES_DIR))

# 1. importe os módulos (não as funções)
import prefilter
import embedder
import help
import prompt_builder

# 2. recarregue apenas os módulos que você editou
reload(prefilter)
reload(embedder)
reload(help)
reload(prompt_builder)

# 3. agora sim importe as funções atualizadas
from prefilter import prefilter_results
from embedder import HuggingFaceEmbedder
from help import estimate_tokens
from prompt_builder import build_prompt


In [63]:
load_dotenv()

True

# First Prompts: Normal RAG + Normal RAG prompting

### Get retrieval data

In [16]:
USER = os.getenv("POSTGRES_USER")
PASSWORD = os.getenv("POSTGRES_PASSWORD")
DB = os.getenv("POSTGRES_DB")
PORT = os.getenv("POSTGRES_PORT")
HOST = os.getenv("POSTGRES_HOST", "localhost")

# Criar engine SQLAlchemy
engine = create_engine(f"postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB}")


In [17]:
df = pd.read_sql("SELECT * FROM retrieved_news_ddgo", engine)

### Filter Data with 20 correspondances

In [19]:
df_filt = df[df['shuffle_id'].between(0, 1999)]

In [20]:
df_filt['search_title'].nunique()

2000

In [21]:
df_filt.groupby('shuffle_id').size().describe()

count    2000.000000
mean       19.876500
std         1.060099
min         2.000000
25%        20.000000
50%        20.000000
75%        20.000000
max        20.000000
dtype: float64

### Filter retrievel data

In [22]:
embedder = HuggingFaceEmbedder(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [25]:
summary_rows = []
filtered_by_title = {}

for title in df_filt['search_title'].unique():


    subset = df_filt[df_filt['search_title'] == title]
    total_before = len(subset)
    subset = subset.drop_duplicates(subset=["original_title"])
    total_after = len(subset)
    duplicates_removed = total_before - total_after
    shuffle_id_val = subset["shuffle_id"].iloc[0]
    results = [
        {
            "refined_title": row["refined_title"],
            "original_title": row["original_title"],
            "domain": row["domain"],
            "search_title": row["search_title"],
            "shuffle_id": row["shuffle_id"],
        }
        for _, row in subset.iterrows()
    ]

    # executa o filtro
    filtered_results = prefilter_results(
        results=results,
        original_title=title,
        embedder=embedder,
        credible_domains_file='../../out/credible_sources.txt'
    )

    # salva resultados completos
    filtered_by_title[title] = filtered_results

    # adiciona linha ao summary
    summary_rows.append({
        "search_title": title,
        "shuffle_id": shuffle_id_val,
        "total_before": total_before,
        "unique_results": total_after,
        "duplicates_removed": duplicates_removed,
        "filtered_results": len(filtered_results),
        "non_filtered_results": total_after - len(filtered_results)
    })

summary_df = pd.DataFrame(summary_rows)

In [26]:
summary_df['filtered_results'].describe()

count    2000.000000
mean        6.282500
std         4.110492
min         1.000000
25%         3.000000
50%         6.000000
75%         9.000000
max        20.000000
Name: filtered_results, dtype: float64

In [27]:
summary_df.to_sql(
    "retrieved_news_ddgo_prefilter_stats",
    engine,
    if_exists="replace",
    index=False
)

1000

### Build Prompts

In [64]:
def generate_prompts(filtered_by_title, mode='test1'):
    rows = []
    for title, filtered_results in filtered_by_title.items():

        shuffle_id = filtered_results[0].get("shuffle_id") if filtered_results else None

        prompt = build_prompt(
            mode=mode,
            title_to_check=title,
            results_filtered=filtered_results
        )

        info = {
            "search_title": title,
            "shuffle_id": shuffle_id,
            "prompt": prompt,
            "num_results": len(filtered_results),
            "approx_tokens": estimate_tokens(prompt),
            "prompt_length_chars": len(prompt),
        }
        rows.append(info)

    df_prompts = pd.DataFrame(rows)
    return df_prompts


In [69]:
df_test1 = generate_prompts(filtered_by_title, mode="test1")

df_test1.to_sql(
    "test1_prompts",
    engine,
    if_exists="replace",   # substitui staging a cada execução
    index=False
)

1000

In [70]:
df_test2 = generate_prompts(filtered_by_title, mode="test2")

df_test2.to_sql(
    "test2_prompts",
    engine,
    if_exists="replace",   # substitui staging a cada execução
    index=False
)

1000

In [ ]:
df_test3 = generate_prompts(filtered_by_title, mode="test3")

df_test3.to_sql(
    "test3_prompts",
    engine,
    if_exists="replace",   # substitui staging a cada execução
    index=False
)